# LSTM

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

2024-11-20 22:49:43.283719: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-20 22:49:43.285595: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-20 22:49:43.295479: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-20 22:49:43.305782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732139383.330894   67632 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732139383.33

In [2]:
file_path = 'data/'

X_train = pd.read_csv(file_path + 'train_f_x.csv') #  entre le 1er janvier 2015 et le 31 décembre 2022 (soit 2922 jours)
y_train = pd.read_csv(file_path + 'y_train_sncf.csv', index_col=0)
X_test = pd.read_csv(file_path + 'test_f_x_THurtzP.csv', index_col=0) # entre le 1er janvier 2023 et le 30 juin 2023 (soit 181 jours) 

X_train['index'] = X_train['date'].astype(str) + '_' + X_train['station'].astype(str)
X_train = X_train.set_index('index')

X_train['date'] = pd.to_datetime(X_train['date'])
X_test['date'] = pd.to_datetime(X_test['date'])

In [3]:
def create_features(df):
    ''' Create features from date '''
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofweek'] = df['date'].dt.dayofweek
    return df

In [ ]:
def add_lags(df_x, df_y):
    ''' Add lags '''
    target_map = df_y['y'].to_dict()

    df_x['lagDay'] = (
        (df_x.date - pd.DateOffset(days=1))
        .dt.strftime('%Y-%m-%d') + '_' + df_x.station
    ).map(target_map)
    
    df_x['lagWeek'] = (
        (df_x.date - pd.DateOffset(days=7))
        .dt.strftime('%Y-%m-%d') + '_' + df_x.station
    ).map(target_map)

    return df_x

In [5]:
X_train = create_features(X_train)
X_train = add_lags(X_train, y_train)
X_test = create_features(X_test)
X_test = add_lags(X_test, y_train)

In [6]:
# Filtrar apenas as features e variáveis alvo
features = ['lagDay', 'lagWeek', 'day', 'month', 'year', 'dayofweek', 'job', 'ferie', 'vacances']
X_train = X_train[features]
X_test = X_test[features]
y_train = y_train['y']  # variável alvo

# Normalizar os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converter para o formato 3D necessário para LSTM (samples, timesteps, features)
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Construir o modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))  # Para reduzir overfitting
model.add(Dense(1))  # Saída única para prever o valor de 'y'

# Compilar o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=2)

W0000 00:00:1732139430.213153   67632 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/isarigue/miniconda3/envs/stasc/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
38434/38434 - 73s - 2ms/step - loss: nan
Epoch 2/20
38434/38434 - 67s - 2ms/step - loss: nan
Epoch 3/20
38434/38434 - 79s - 2ms/step - loss: nan
Epoch 4/20
38434/38434 - 49s - 1ms/step - loss: nan
Epoch 5/20
38434/38434 - 36s - 940us/step - loss: nan
Epoch 6/20
38434/38434 - 38s - 978us/step - loss: nan
Epoch 7/20
38434/38434 - 37s - 969us/step - loss: nan
Epoch 8/20
38434/38434 - 37s - 973us/step - loss: nan
Epoch 9/20
38434/38434 - 41s - 1ms/step - loss: nan
Epoch 10/20
38434/38434 - 41s - 1ms/step - loss: nan
Epoch 11/20
38434/38434 - 41s - 1ms/step - loss: nan
Epoch 12/20
38434/38434 - 40s - 1ms/step - loss: nan
Epoch 13/20
38434/38434 - 36s - 950us/step - loss: nan
Epoch 14/20
38434/38434 - 37s - 975us/step - loss: nan
Epoch 15/20
38434/38434 - 37s - 970us/step - loss: nan
Epoch 16/20
38434/38434 - 41s - 1ms/step - loss: nan
Epoch 17/20
38434/38434 - 44s - 1ms/step - loss: nan
Epoch 18/20
38434/38434 - 39s - 1ms/step - loss: nan
Epoch 19/20
38434/38434 - 38s - 990us/ste

In [ ]:
# Predict using the trained model
y_pred = pd.DataFrame(index=X_test.index)
y_pred_vals = model.predict(X_test)
y_pred['y'] = np.clip(y_pred_vals, 0, None).astype(int) # Clip negative values to 0
y_pred.to_csv(file_path + 'y_pred_lstm.csv')

In [10]:
X_test_aux = pd.read_csv(file_path + 'test_f_x_THurtzP.csv', index_col=0) 

# Predict using the trained model
y_pred = pd.DataFrame(index=X_test_aux.index)
y_pred_vals = model.predict(X_test)
y_pred['y'] = np.clip(y_pred_vals, 0, None).astype(int) # Clip negative values to 0
y_pred.to_csv(file_path + 'y_pred_lstm.csv')

# Avaliar o modelo
#from sklearn.metrics import mean_absolute_percentage_error
# Se tiver y_test, pode-se usar: mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"Previsões para o conjunto de teste: {y_pred}")

2458/2458 ━━━━━━━━━━━━━━━━━━━━ 2s 739us/step
Previsões para o conjunto de teste:                                   y
index                              
2023-01-01_1J7 -9223372036854775808
2023-01-01_O2O -9223372036854775808
2023-01-01_8QR -9223372036854775808
2023-01-01_L58 -9223372036854775808
2023-01-01_UMC -9223372036854775808
...                             ...
2023-05-31_N9K -9223372036854775808
2023-05-31_P6E -9223372036854775808
2023-05-31_BDC -9223372036854775808
2023-05-31_QD6 -9223372036854775808
2023-05-31_W14 -9223372036854775808

[78652 rows x 1 columns]


/tmp/ipykernel_67632/1223165092.py:6: RuntimeWarning: invalid value encountered in cast
  y_pred['y'] = np.clip(y_pred_vals, 0, None).astype(int) # Clip negative values to 0
